# ML Preprocessing
Steve Donahue, www.github.com/sdonahue0132

This notebook uses functions similar to those found in the EDA + Simulation Data Preprocessing notebook.  When run, it produces additional data that can be used for additional performance evaluation for the ML classifiers.

In [ ]:
# Import essential libraries #

import pandas as pd
import numpy as np
from numpy import percentile
import math
import datetime
from datetime import date
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from functools import reduce
import warnings
from random import randint
import seaborn as sns
%matplotlib inline

warnings.filterwarnings(action='once')

# Preprocessing for Application

In [3]:
# Import VIGRX, define functions to handle weekly moving averages and multiweek moving averages

def CSV_Formatter (folder_name, filename):
    df_init = pd.read_csv('raw_data/' + str(folder_name) +'/' + str(filename) +'.csv')
    df = df_init.set_index('Date')[['Close']]
    df.columns = [str(filename)]
    df['Next Day Values'] = df[filename].shift(-1)
    df['Percentages'] = df['Next Day Values']/df[str(filename)]
    df_final = df[['Percentages']]
    df_final.columns = [str(filename)]
    return(df[[filename]], df_final)
    
prices, percentages = CSV_Formatter('Stock_Indices', 'VIGRX')

def weekly_master(df, cutoff_date):
    
# A little setup is necessary to ensure fidelity across weekly data.  In the original yahoo finance data downloads,
# Weekends and holidays are not counted in the Datetime Index.  The following code creates a working df that
# Can be broken into calendar weeks at regular 7 day intervals, to better reflect paycheck contributions and 
# subsequent analyses can be done on a week to week basis.

# Note that for days where data is unavailable, I've filled in the value 1, since the method of assessing portfolios
# Is multiplication across daily percentage changes.  In this way, days when no percent changes are documented do not
# affect the value of the investment.

    # NOTE THAT THIS FUNCTION WILL ONLY ACCOMODATE DATAFRAMES WITH UP TO 8 COLUMNS AS WRITTEN! #

    cutoff_date = pd.to_datetime(cutoff_date)
    cutoff = cutoff_date - datetime.timedelta(days=cutoff_date.weekday())
    
    if cutoff_date.weekday() == 6:
        td = timedelta(1)
        cutoff_date = cutoff_date - td
    
    datelist = pd.to_datetime(df.index.values)
    df['Datetime'] = datelist
    df_timed = df.set_index('Datetime')
    labels = df_timed.columns

    df_segmented = pd.DataFrame(columns = labels)

    daterange = int(str(df_timed.index.max() - df_timed.index.min()).replace(" days 00:00:00", ''))
    all_dates = pd.date_range(df_timed.index.min(), periods=daterange).tolist()
    
    index_df = pd.DataFrame(all_dates)
    index_df.columns = ['Datetime']
    
    working_df = index_df.merge(df_timed, how = 'outer', left_on = 'Datetime', right_on ='Datetime')
    working_df = working_df.fillna(1).set_index('Datetime').sort_values('Datetime', ascending = False)
    working_df = working_df.loc[working_df.index <= cutoff]

    days = len(working_df.index)
    number_of_weeks = int(np.floor(days/7))

    df_progress = pd.DataFrame(index=[0,1,2,3,4])
    weekly_eval = pd.DataFrame()

    for i in range(0, number_of_weeks):
        portfolio_segment = working_df.iloc[i*7+1:(i+1)*7]
        
        products =[]
        prod_1 = portfolio_segment.iloc[:, 0].product()
        products.append(prod_1)

        if len(labels) > 1:
            prod_2 = portfolio_segment.iloc[:, 1].product()
            products.append(prod_2)
        if len(labels) > 2:        
            prod_3 = portfolio_segment.iloc[:, 2].product()
            products.append(prod_3)
        if len(labels) > 3:
            prod_4 = portfolio_segment.iloc[:, 3].product()
            products.append(prod_4)
        if len(labels) > 4:
            prod_5 = portfolio_segment.iloc[:, 4].product()
            products.append(prod_5)
        if len(labels) > 5:
            prod_6 = portfolio_segment.iloc[:, 5].product()
            products.append(prod_6)
        if len(labels) > 6:
            prod_7 = portfolio_segment.iloc[:, 6].product()
            products.append(prod_7)
        if len(labels) > 7:
            prod_8 = portfolio_segment.iloc[:, 7].product()
            products.append(prod_8)
   
    # Can we just make this into a loop?
    # For j in range(0, len(labels)):
    #    temp_prod = portfolio_segment.iloc[:, j].product()
    #    products.append(temp_prod)

        td = timedelta(1)
        weekly_eval[str(portfolio_segment.index[(cutoff + td).weekday()]).replace("00:00:00", '')] = products
    
    weekly_eval = weekly_eval.T
    weekly_eval.columns = labels
    weekly_eval.index = pd.to_datetime(weekly_eval.index)
    weekly_eval.index.name = 'Date'
    weekly_eval.sort_index()

    return(weekly_eval)

def timed_eval(keyword, df, num):

    title = keyword + ' ' + str(num) + '_Week_Avg' 
    moving_avg = []

    for i in range(int(round(df.shape[0]))):
        selection = df.iloc[i+1: i+ 1 + num, :]
        prod = selection.iloc[:, 0].product()
        moving_avg = np.append(moving_avg, prod)
    
    df[title]= moving_avg
    return(df)


In [4]:
# Fidelity US 500, 'FUSEX', was rolled into another index, 'FXAIX' in November 2018.  
# This cell joins the two records at the transition point for.... fidelity.

(FXAIX, FXAIX_final) = CSV_Formatter('Stock_Indices', 'FXAIX')
(FUSEX, FUSEX_final) = CSV_Formatter('Stock_Indices', 'FUSEX')

FUSEX = pd.merge( FUSEX_final, FXAIX_final, how = 'outer', on = 'Date')

FUSEX = pd.DataFrame(FUSEX['FUSEX'].fillna(FUSEX['FXAIX']))



In [5]:
# This Cell reads all necessary source files for STOCK INDICES #

(PREIX, PREIX_final) = CSV_Formatter('Stock_Indices', 'PREIX')
(SWPPX, SWPPX_final) = CSV_Formatter('Stock_Indices', 'SWPPX')
#(FUSEX, FUSEX_final) = CSV_Formatter('Stock_Indices', 'FUSEX_plus')
(VFINX, VFINX_final) = CSV_Formatter('Stock_Indices', 'VFINX')
(VIGRX, VIGRX_final) = CSV_Formatter('Stock_Indices', 'VIGRX')


# Creates dataframes of their daily values and daily percent changes , aka increments.
dfstock_values = [PREIX[['PREIX']], SWPPX[['SWPPX']], FUSEX[['FUSEX']], VFINX[['VFINX']], VIGRX[['VIGRX']]]
dfstock_finals = [PREIX_final, SWPPX_final, FUSEX, VFINX_final, VIGRX_final]


#stock_daily_values_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), dfstock_values)
stock_increments_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), dfstock_finals)

pd.merge(stock_increments_df, FUSEX_final, how = 'outer')
stock_weekly = weekly_master(stock_increments_df, '2019-02-01')


In [6]:
df = stock_weekly

df = df[['PREIX', 'SWPPX', 'FUSEX', 'VFINX']]

df['Average'] = (df['PREIX'] + df['SWPPX'] + df['FUSEX'] + df['VFINX'])/4

stocks = pd.DataFrame()
stocks.index_name = df.index.name
stocks['Stock_increments'] = df.Average

stocks.index = df.index

stocks_2 = timed_eval('Stocks', stocks, 2)
stocks_3 = timed_eval('Stocks', stocks_2, 3)
stocks_6 = timed_eval('Stocks', stocks_3, 6)
stocks_9 = timed_eval('Stocks', stocks_6, 9)
stocks_12 = timed_eval('Stocks', stocks_9, 12)
stocks_df = timed_eval('Stocks', stocks_12, 15)

In [7]:
# This Cell reads all necessary source files for Intermediate Term Bonds #

(BIV, BIV_final) = CSV_Formatter('Intermediate_Bonds', 'BIV')
(HYG, HYG_final) = CSV_Formatter('Intermediate_Bonds', 'HYG')
(IEF, IEF_final) = CSV_Formatter('Intermediate_Bonds', 'IEF')
(IEI, IEI_final) = CSV_Formatter('Intermediate_Bonds', 'IEI')
(IGIB, IGIB_final) = CSV_Formatter('Intermediate_Bonds', 'IGIB')
(IPE, IPE_final) = CSV_Formatter('Intermediate_Bonds', 'IPE')
(ITE, ITE_final) = CSV_Formatter('Intermediate_Bonds', 'ITE')
(TIP, TIP_final) = CSV_Formatter('Intermediate_Bonds', 'TIP')

# Creates dataframes of their daily values and daily percent changes , aka increments.
df_itb_values = [BIV[['BIV']], HYG[['HYG']], IEF[['IEF']], IEI[['IEI']], IGIB[['IGIB']], IPE[['IPE']], ITE[['ITE']], TIP[['TIP']]]
df_itb_finals = [BIV_final, HYG_final, IEF_final, IEI_final, IGIB_final, IPE_final, ITE_final, TIP_final]
itb_daily_values_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), df_itb_values)
itb_increments_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), df_itb_finals)

itb_weekly = weekly_master(itb_increments_df, '2019-02-01')


In [8]:
df = itb_weekly

df = df[['BIV', 'HYG', 'IEF', 'IEI', 'IGIB', 'IPE', 'ITE', 'TIP']]

df['Average'] = (df['BIV'] + df['HYG'] + df['IEF'] + df['IEI'] + df['IGIB'] + df['IPE'] + df['ITE'] + df['TIP'])/8

itb = pd.DataFrame()

itb.index_name = df.index.name
itb['Itb_increments'] = df.Average
itb.index = df.index

itb_2 = timed_eval('ITB', itb, 2)
itb_3 = timed_eval('ITB', itb_2, 3)
itb_6 = timed_eval('ITB', itb_3, 6)
itb_9 = timed_eval('ITB', itb_6, 9)
itb_12 = timed_eval('ITB', itb_9, 12)
itb_df = timed_eval('ITB', itb_12, 15)

In [9]:
# This Cell reads all necessary source files for Long Term Bonds #

(PRULX, PRULX_final) = CSV_Formatter('Long_Term_Bonds', 'PRULX')
(VUSTX, VUSTX_final) = CSV_Formatter('Long_Term_Bonds', 'VUSTX')
(WHOSX, WHOSX_final) = CSV_Formatter('Long_Term_Bonds', 'WHOSX')

# Creates dataframes of their daily values and daily percent changes , aka increments.
df_ltb_values = [PRULX[['PRULX']], VUSTX[['VUSTX']], WHOSX[['WHOSX']]]
df_ltb_finals = [PRULX_final, VUSTX_final, WHOSX_final]
ltb_daily_values_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), df_ltb_values)
ltb_increments_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), df_ltb_finals)

ltb_weekly = weekly_master(ltb_increments_df, '2019-02-01')


In [10]:
df = ltb_weekly

df = df[['PRULX', 'VUSTX', 'WHOSX']]

df['Average'] = (df['PRULX'] + df['VUSTX'] + df['WHOSX'])/3

ltb = pd.DataFrame()

ltb.index_name = df.index.name
ltb['Ltb_increments'] = df.Average
ltb.index = df.index

ltb_2 = timed_eval('LTB', ltb, 2)
ltb_3 = timed_eval('LTB', ltb_2, 3)
ltb_6 = timed_eval('LTB', ltb_3, 6)
ltb_9 = timed_eval('LTB', ltb_6, 9)
ltb_12 = timed_eval('LTB', ltb_9, 12)
ltb_df = timed_eval('LTB', ltb_12, 15)

In [11]:
# This Cell reads all necessary source files for Gold 

(INIVX, INIVX_final) = CSV_Formatter('Gold', 'INIVX')
(OPGSX, OPGSX_final) = CSV_Formatter('Gold', 'OPGSX')
(SGGDX, SGGDX_final) = CSV_Formatter('Gold', 'SGGDX')
(USERX, USERX_final) = CSV_Formatter('Gold', 'USERX')
(VGPMX, VGPMX_final) = CSV_Formatter('Gold', 'VGPMX')

# Creates dataframes of their daily values and daily percent changes , aka increments.
dfgold_values = [INIVX[['INIVX']], OPGSX[['OPGSX']], SGGDX[['SGGDX']], USERX[['USERX']], VGPMX[['VGPMX']]]
dfgold_finals = [INIVX_final, OPGSX_final, SGGDX_final, USERX_final, VGPMX_final]
gold_daily_values_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), dfgold_values)
gold_increments_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), dfgold_finals)

gold_weekly = weekly_master(gold_increments_df, '2019-02-01')


In [12]:
df = gold_weekly

df = df[['INIVX', 'OPGSX', 'SGGDX', 'USERX', 'VGPMX']]

df['Average'] = (df['INIVX'] + df['OPGSX'] + df['SGGDX'] + df['USERX'] + df['VGPMX'])/5

gold = pd.DataFrame()

gold.index_name = df.index.name
gold['Gold_increments'] = df.Average
gold.index = df.index

gold_2 = timed_eval('Gold', gold, 2)
gold_3 = timed_eval('Gold', gold_2, 3)
gold_6 = timed_eval('Gold', gold_3, 6)
gold_9 = timed_eval('Gold', gold_6, 9)
gold_12 = timed_eval('Gold', gold_9, 12)
gold_df = timed_eval('Gold', gold_12, 15)

In [13]:
# This Cell reads all necessary source files from their respective download folders for Broad Basket Commodities #

(DBC, DBC_final) = CSV_Formatter('Broad_Commodities', 'DBC')
(DJP, DJP_final) = CSV_Formatter('Broad_Commodities', 'DJP')
(GSG, GSG_final) = CSV_Formatter('Broad_Commodities', 'GSG')
(GSP, GSP_final) = CSV_Formatter('Broad_Commodities', 'GSP')

# Creates dataframes of their daily values and daily percent changes , aka increments.
df_commod_values = [DBC[['DBC']], DJP[['DJP']], GSG[['GSG']], GSP[['GSP']]]
df_commod_finals = [DBC_final, DJP_final, GSG_final, GSP_final]
commod_daily_values_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), df_commod_values)
commod_increments_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), df_commod_finals)

commod_weekly = weekly_master(commod_increments_df, '2019-02-01')


In [14]:
df = commod_weekly

df = df[['DBC', 'DJP', 'GSG', 'GSP']]

df['Average'] = (df['DBC'] + df['DJP'] + df['GSG'] + df['GSP'])/4

commod = pd.DataFrame()

commod.index_name = df.index.name
commod['Commodity_increments'] = df.Average
commod.index = df.index

commod_2 = timed_eval('Commod', commod, 2)
commod_3 = timed_eval('Commod', commod_2, 3)
commod_6 = timed_eval('Commod', commod_3, 6)
commod_9 = timed_eval('Commod', commod_6, 9)
commod_12 = timed_eval('Commod', commod_9, 12)
commod_df = timed_eval('Commod', commod_12, 15)


In [15]:
from functools import reduce

dfs = [stocks_df, itb_df, ltb_df, gold_df, commod_df]

for df in dfs:
    df.index = pd.to_datetime(df.index)

df_final = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)

full_features = df_final

# Producing recommendation values using Tuned ML 

In [16]:
prices, percentages = CSV_Formatter('Stock_Indices', 'VIGRX')

weeks = weekly_master(percentages, '2019-02-01')
weeks_2 = timed_eval('VIGRX', weeks, 2)
weeks_3 = timed_eval('VIGRX', weeks_2, 3)
weeks_6 = timed_eval('VIGRX', weeks_3, 6)
weeks_9 = timed_eval('VIGRX', weeks_6, 9)
weeks_12 = timed_eval('VIGRX', weeks_9, 12)
weeks_df = timed_eval('VIGRX', weeks_12, 15)


In [17]:
historical = pd.merge(df_final, weeks_df, on='Date')

#historical.to_csv('csv_files/historical_record.csv')